# TP 4 Enmascaramiento de Video   

## Librerias necesarias

In [ ]:
import cv2
import numpy as np

# Implementación de la solución

In [ ]:
# Parametros
N = 30  # Numero de frames para la estimación de la media
update_interval = 100  # Actualizamos el background cada 100 frames

def median_background_estimator(video_path, N, update_interval):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fg_masks = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray)
        frame_count += 1

        if len(frames) > N:
            frames.pop(0)

        if frame_count % update_interval == 0 or frame_count <= N:
            # Computamos la media del background
            background = np.median(np.array(frames), axis=0).astype(np.uint8)

        # Compuatamos la foreground mask
        fg_mask = cv2.absdiff(gray, background)
        _, fg_mask = cv2.threshold(fg_mask, 25, 255, cv2.THRESH_BINARY)
        fg_masks.append(fg_mask)

        # Aplicamos la máscara al frame
        fg_segmented = cv2.bitwise_and(frame, frame, mask=fg_mask)

        # Mostramos la salida segmentada con cv2.imshow
        cv2.imshow('Segmented Frame', fg_segmented)
        cv2.imshow('Foreground Mask', fg_mask)
        cv2.imshow('Original Frame', frame)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def gmm_background_subtractor(video_path):
    cap = cv2.VideoCapture(video_path)
    fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        fg_mask = fgbg.apply(frame)

        # Aplicamos la máscara al frame
        fg_segmented = cv2.bitwise_and(frame, frame, mask=fg_mask)

        # Mostramos la salida segmentada con cv2.imshow
        cv2.imshow('Segmented Frame', fg_segmented)
        cv2.imshow('Foreground Mask', fg_mask)
        cv2.imshow('Original Frame', frame)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Ejecución

In [5]:
# Path  al archivo de video
video_path = 'video/vtest.avi'

# Detector de Background Naive con Estimation de media
median_background_estimator(video_path, N=N, update_interval=update_interval)

# Subtractor de Background GMM-based
gmm_background_subtractor(video_path)
